# Manipulating the data

In a <a href="https://github.com/edu9as/random-notebooks/blob/main/NBAPlayOffs/PreparingTheData.ipynb">previous notebook</a> I got a table from BasketballReference website and, after some basic formatting (renaming columns, removing empty rows and columns), the dataset was splitted according to the league of each PlayOffs series. The result were three CSV files: ```aba.csv```, ```baa.csv``` and ```nba.csv```.

In this notebook, I am using data wrangling methods to clean the data in ```nba.csv```. These methods can be applied to the dataframes in the other two files, as the original structure of the data is the same.

Let's load the dataframe inside this ```nba.csv``` file and see how it looks:

In [2]:
import pandas as pd

nba = pd.read_csv("nba.csv")
nba.head()

,Year,Series,Date,Team_win,Wins_win,Team_lose,Wins_lose,Fav,Underdog
0,2020,Eastern Conf First Round,"Aug 18 - Aug 29, 2020",Milwaukee Bucks (1),4,Orlando Magic (8),1,MIL (-7500),ORL (+3250)
1,2020,Eastern Conf First Round,"Aug 17 - Aug 23, 2020",Toronto Raptors (2),4,Brooklyn Nets (7),0,TOR (-2200),BRK (+1315)
2,2020,Eastern Conf First Round,"Aug 17 - Aug 23, 2020",Boston Celtics (3),4,Philadelphia 76ers (6),0,BOS (-450),PHI (+360)
3,2020,Eastern Conf First Round,"Aug 18 - Aug 24, 2020",Miami Heat (5),4,Indiana Pacers (4),0,MIA (-320),IND (+260)
4,2020,Western Conf First Round,"Aug 18 - Aug 29, 2020",Los Angeles Lakers (1),4,Portland Trail Blazers (8),1,LAL (-550),POR (+425)


I see tons of possibilities to improve the representation of the data in this dataframe. Let's discuss it column by column:

- **Year**: integer type column, I think there is not anything to improve here by the moment.
- **Series**: I think this column might be much more useful if it were splitted in two columns, i.e., 
    - **Series_conference**: categorical type column, three unique values (**E** for Eastern, **W** for Western and **F** for NBA Finals).
    - **Series_round**: categorical type column, four distinct values (**0** for NBA Finals, **1** for Conference Finals, **2** for Conference Semifinals and **3** for First Round). The reason why I have decided to encode the levels in this order is because the number of First Round series has changed with time.
- **Date**: I prefer these values to be ```datetime``` rather than string. Also, I thing splitting them in **Date_start** and **Date_end** would be a good idea.
- **Team_win** and **Team_lose**: in both columns, the name of the team is before the parenthesis and the seed of the team (i.e., the ranking of the team in its conference after regular season) is found between parenthesis. Then, these columns should be splitted in:
    - **Team_win** and **Seed_win**
    - **Team_lose** and **Seed_lose**
- **Wins_win** and **Wins_lose** represent the games won by the team who won the series and the number of games won by the team who won the series, respectively. I think nothing can be improved in these columns by the moment.
- **Fav** and **Underdog**: these columns give information about the team who was the favorite for winning the series right before they took place, and the odds between parenthesis. I am thinking of splitting them in **Fav_team** and **Fav_odds**, and **Underdog_team** and **Underdog_odds**.

Then, let's go step by step.

## Reformatting Series column

Let's inspect the data in **Series** column:

In [4]:
nba.Series.value_counts()

Eastern Conf First Round          164
Western Conf First Round          164
Eastern Conf Semifinals           100
Western Conf Semifinals           100
Finals                             71
Eastern Conf Finals                50
Western Conf Finals                50
Western Div Semifinals             28
Eastern Div Semifinals             28
Western Div Finals                 21
Eastern Div Finals                 21
Central Div Semifinals              2
Western Div Tiebreak                2
Eastern Div 3rd Place Tiebreak      1
Central Div Finals                  1
Semifinals                          1
Western Div 2nd Place Tiebreak      1
Central Div 1st Place Tiebreak      1
Central Div 3rd Place Tiebreak      1
Name: Series, dtype: int64

We see there are some strange series that are not Conference-wise but Division-wise. These series occured before 1971, season when the current NBA PlayOffs format was adopted. Then, I am splitting the database into **old_nba** and **nba** dataframes.

In [6]:
old_nba = nba[nba.Year <= 1970]
nba = nba[nba.Year > 1970]

Now the data in **Series** column is more homogeneous:

In [7]:
nba.Series.value_counts()

Eastern Conf First Round    164
Western Conf First Round    164
Eastern Conf Semifinals     100
Western Conf Semifinals     100
Finals                       50
Eastern Conf Finals          50
Western Conf Finals          50
Name: Series, dtype: int64

In [8]:
import warnings

warnings.filterwarnings('ignore')

In [9]:
import numpy as np

nba["Series_conference"] = np.where([("Eastern" in v) 
                                     for v in nba.Series.values], 
                                    "E",  # E for Eastern series
                                    np.where([("Western" in v) 
                                              for v in nba.Series.values], 
                                             "W",  # W for Western series
                                             "F"))  # F for NBA Finals series

nba["Series_round"] = np.where([("First" in v) for v in nba.Series.values], 
                               3,  # First Round encoder as 3
                               np.where([("Semifinals" in v) 
                                         for v in nba.Series.values], 
                                        2,  # Conference semifinals encoded as 2
                                        np.where([("Finals" == v) 
                                        for v in nba.Series.values], 
                                        0,  # NBA Finals encoded as 0
                                        1)))  # Conference Finals encoded as 1

Let's explore the result:

In [10]:
import random

rand_idx = random.sample(range(len(nba)), k = 10)

nba[["Series", "Series_conference", "Series_round"]].iloc[rand_idx]

,Series,Series_conference,Series_round
501,Western Conf First Round,W,3
216,Western Conf First Round,W,3
665,Eastern Conf Semifinals,E,2
298,Western Conf Finals,W,1
114,Eastern Conf Semifinals,E,2
53,Eastern Conf Semifinals,E,2
289,Western Conf First Round,W,3
502,Western Conf First Round,W,3
416,Western Conf Semifinals,W,2
393,Eastern Conf First Round,E,3


## Date column

The strategy here is to capture the start and end date of each PlayOffs series with two simple regular expressions:
- Start date is a three-letters word followed by a space followed by one or more digits. The year (4-digits number) is also captured, preceded by a blank space. Then, one valid regular expression is ```"(\w{3}\s\d+).*(\s\d{4})"```.
- End date is very similar to start date, but there are some letters before the end date. Then, one possibility is ```"\w.*(\w{3}\s\d+).*(\s\d{4})"```.

In [11]:
import re

start = "(\w{3}\s\d+).*(\s\d{4})"
end = "\w.*(\w{3}\s\d+).*(\s\d{4})"

nba["Date_start"] = [pd.to_datetime("".join(re.search(start, d).groups())) 
                     for d in nba.Date.values]
nba["Date_end"] = [pd.to_datetime("".join(re.search(end, d).groups())) 
                   for d in nba.Date.values]

As a result:

In [12]:
nba[["Date", "Date_start", "Date_end"]].iloc[rand_idx]

,Date,Date_start,Date_end
501,"Apr 23 - Apr 29, 1987",1987-04-23,1987-04-29
216,"Apr 23 - May 6, 2006",2006-04-23,2006-05-06
665,"Mar 31 - Apr 11, 1972",1972-03-31,1972-04-11
298,"May 19 - May 27, 2001",2001-05-19,2001-05-27
114,"May 5 - May 18, 2013",2013-05-05,2013-05-18
53,"May 1 - May 7, 2017",2017-05-01,2017-05-07
289,"Apr 21 - May 3, 2001",2001-04-21,2001-05-03
502,"Apr 23 - Apr 30, 1987",1987-04-23,1987-04-30
416,"May 10 - May 22, 1993",1993-05-10,1993-05-22
393,"Apr 29 - May 6, 1994",1994-04-29,1994-05-06


## Team_win and Team_lose

The strategy is the same: capture the team and the seed with regular expressions.

In [13]:
nba["Seed_win"] = [int(re.search("\((\d)", n).group(1)) 
                   for n in nba.Team_win.values]
nba["Team_win"] = [re.search("(.*)\s\(", n).group(1) 
                   for n in nba.Team_win.values]
                   
nba["Seed_lose"] = [int(re.search("\((\d)", n).group(1)) 
                    for n in nba.Team_lose.values]
nba["Team_lose"] = [re.search("(.*)\s\(", n).group(1) 
                    for n in nba.Team_lose.values]


In [14]:
nba[["Team_win", "Seed_win", "Team_lose", "Seed_lose"]].iloc[rand_idx]

,Team_win,Seed_win,Team_lose,Seed_lose
501,Los Angeles Lakers,1,Denver Nuggets,8
216,Phoenix Suns,2,Los Angeles Lakers,7
665,New York Knicks,2,Baltimore Bullets,1
298,Los Angeles Lakers,2,San Antonio Spurs,1
114,Indiana Pacers,3,New York Knicks,2
53,Cleveland Cavaliers,2,Toronto Raptors,3
289,Dallas Mavericks,5,Utah Jazz,4
502,Seattle SuperSonics,7,Dallas Mavericks,2
416,Seattle SuperSonics,3,Houston Rockets,2
393,New York Knicks,2,New Jersey Nets,7


In [15]:
nba.iloc[372]

Year                                  1996
Series                 Eastern Conf Finals
Date                 May 19 - May 27, 1996
Team_win                     Chicago Bulls
Wins_win                                 4
Team_lose                    Orlando Magic
Wins_lose                                0
Fav                             CHI (-500)
Underdog                        ORL (+350)
Series_conference                        E
Series_round                             1
Date_start             1996-05-19 00:00:00
Date_end               1996-05-27 00:00:00
Seed_win                                 1
Seed_lose                                2
Name: 372, dtype: object

## Fav and Underdog

Finally, let's separate the name of the favorite and underdog teams (indeed, this is an abbreviation, not the real name of the team) from the odds of the teams winning the series. We will also achieve this goal with regular expressions.

In [16]:
for col in ("Fav", "Underdog"):
    # Filling empty values with "NA (0)"
    nba[col] = np.where(nba[col].isnull(),
                        "NA (0)", 
                        nba[col])
    
    # Capturing the data
    nba[col + "_odds"] = [int(re.search("\(([\-+\d]*)", n).group(1)) 
                          for n in nba[col].values]
    nba[col + "_team"] = [re.search("(.*)\s\(", n).group(1) 
                          for n in nba[col].values]

Does the favorite team always win the series?

In [17]:
nba[["Team_win", "Fav_team", "Fav_odds", 
     "Underdog_team", "Underdog_odds"]].iloc[rand_idx]

,Team_win,Fav_team,Fav_odds,Underdog_team,Underdog_odds
501,Los Angeles Lakers,NA,0,NA,0
216,Phoenix Suns,PHO,-360,LAL,300
665,New York Knicks,NA,0,NA,0
298,Los Angeles Lakers,LAL,-145,SAS,125
114,Indiana Pacers,NYK,-190,IND,165
53,Cleveland Cavaliers,CLE,-475,TOR,380
289,Dallas Mavericks,UTA,-180,DAL,160
502,Seattle SuperSonics,NA,0,NA,0
416,Seattle SuperSonics,SEA,-135,HOU,115
393,New York Knicks,NYK,-360,NJN,280


## The End

The final table looks like this:

In [18]:
pretty_cols = [
    "Year",
    "Series_conference",
    "Series_round",
    "Date_start", 
    "Date_end", 
    "Team_win", 
    "Seed_win",
    "Wins_win",
    "Team_lose", 
    "Seed_lose",
    "Wins_lose",
    "Fav_team",
    "Fav_odds",
    "Underdog_team", 
    "Underdog_odds"
]

nba[pretty_cols].iloc[rand_idx]

,Year,Series_conference,Series_round,Date_start,Date_end,Team_win,Seed_win,Wins_win,Team_lose,Seed_lose,Wins_lose,Fav_team,Fav_odds,Underdog_team,Underdog_odds
501,1987,W,3,1987-04-23,1987-04-29,Los Angeles Lakers,1,3,Denver Nuggets,8,0,NA,0,NA,0
216,2006,W,3,2006-04-23,2006-05-06,Phoenix Suns,2,4,Los Angeles Lakers,7,3,PHO,-360,LAL,300
665,1972,E,2,1972-03-31,1972-04-11,New York Knicks,2,4,Baltimore Bullets,1,2,NA,0,NA,0
298,2001,W,1,2001-05-19,2001-05-27,Los Angeles Lakers,2,4,San Antonio Spurs,1,0,LAL,-145,SAS,125
114,2013,E,2,2013-05-05,2013-05-18,Indiana Pacers,3,4,New York Knicks,2,2,NYK,-190,IND,165
53,2017,E,2,2017-05-01,2017-05-07,Cleveland Cavaliers,2,4,Toronto Raptors,3,0,CLE,-475,TOR,380
289,2001,W,3,2001-04-21,2001-05-03,Dallas Mavericks,5,3,Utah Jazz,4,2,UTA,-180,DAL,160
502,1987,W,3,1987-04-23,1987-04-30,Seattle SuperSonics,7,3,Dallas Mavericks,2,1,NA,0,NA,0
416,1993,W,2,1993-05-10,1993-05-22,Seattle SuperSonics,3,4,Houston Rockets,2,3,SEA,-135,HOU,115
393,1994,E,3,1994-04-29,1994-05-06,New York Knicks,2,3,New Jersey Nets,7,1,NYK,-360,NJN,280


I am storing this clean dataframe in a new CSV file, which is called **clean_nba.csv**.

In [19]:
nba[pretty_cols].to_csv("clean_nba.csv", index = False)